<a href="https://colab.research.google.com/github/amlzhrara/stunning-waddle/blob/master/MarketBasketAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df = pd.read_csv("/content/drive/MyDrive/data/transaksi_dqlab_retail.tsv", sep = '\t')

In [3]:
df.head()

,Kode Transaksi,Nama Barang
0,#1,Kaos
1,#1,Shampo Biasa
2,#1,Sepatu Sport merk Z
3,#1,Serum Vitamin
4,#1,Baju Renang Pria Dewasa


In [4]:
df.rename(columns={"Kode Transaksi" : "kode"}, inplace = True)
df.rename(columns={"Nama Barang" : "nama_item"}, inplace = True)

In [5]:
df.head()

,kode,nama_item
0,#1,Kaos
1,#1,Shampo Biasa
2,#1,Sepatu Sport merk Z
3,#1,Serum Vitamin
4,#1,Baju Renang Pria Dewasa


In [6]:
count = df['nama_item'].value_counts() 
print(count) 

Shampo Biasa               2075
Serum Vitamin              1685
Baju Batik Wanita          1312
Baju Kemeja Putih          1255
Celana Jogger Casual       1136
                           ... 
Tali Ban Ikat Pinggang       27
Pelembab                     24
Stripe Pants                 19
Tas Kosmetik                 11
Celana Jeans Sobek Pria       9
Name: nama_item, Length: 69, dtype: int64


In [7]:
count.head(10)

Shampo Biasa                 2075
Serum Vitamin                1685
Baju Batik Wanita            1312
Baju Kemeja Putih            1255
Celana Jogger Casual         1136
Cover Koper                  1086
Sepatu Sandal Anak           1062
Tali Pinggang Gesper Pria    1003
Sepatu Sport merk Z           888
Wedges Hitam                  849
Name: nama_item, dtype: int64

In [8]:
count.tail(10)

Dompet Unisex                 75
Tas Sekolah Anak Perempuan    71
Atasan Baju Belang            56
Hair Dye                      46
Baju Renang Pria Anak-anak    32
Tali Ban Ikat Pinggang        27
Pelembab                      24
Stripe Pants                  19
Tas Kosmetik                  11
Celana Jeans Sobek Pria        9
Name: nama_item, dtype: int64

In [10]:
group_item = df.groupby('kode')['nama_item'].apply(lambda x: list(x))

In [11]:
group_item.head()

kode
#1       [Kaos, Shampo Biasa, Sepatu Sport merk Z, Seru...
#10      [Jeans Jumbo, Celana Pendek Jeans, Kaos, Baju ...
#100     [Hair Dryer, Shampo Biasa, Hair Tonic, Sepatu ...
#1000    [Celana Jeans Sobek Wanita, Celana Pendek Jean...
#1001    [Hair Dryer, Wedges Hitam, Sepatu Sport merk Z...
Name: nama_item, dtype: object

In [14]:
basket = (df.groupby(['kode', 'nama_item'])['nama_item'].count().unstack().reset_index().fillna(0).set_index('kode'))
basket.head()

nama_item,Atasan Baju Belang,Atasan Kaos Putih,Baju Batik Wanita,Baju Kaos Anak - Karakter Kartun,Baju Kaos Anak - Superheroes,Baju Kaos Olahraga,Baju Kemeja Putih,Baju Renang Anak Perempuan,Baju Renang Pria Anak-anak,Baju Renang Pria Dewasa,Baju Renang Wanita Dewasa,Blouse Denim,Celana Jeans Sobek Pria,Celana Jeans Sobek Wanita,Celana Jogger Casual,Celana Panjang Format Hitam,Celana Pendek Casual,Celana Pendek Green/Hijau,Celana Pendek Jeans,Celana Tactical,Cover Koper,Cream Whitening,Dompet Card Holder,Dompet Flip Cover,Dompet Kulit Pria,Dompet STNK Gantungan,Dompet Unisex,Flat Shoes Ballerina,Gembok Koper,Hair Dryer,Hair Dye,Hair Tonic,Hair and Scalp,Jeans Jumbo,Kaos,Koper Fiber,Kuas Makeup,Mascara,Minyak Rambut,Obat Penumbuh Rambut,Pelembab,Sepatu Kulit Casual,Sepatu Sandal Anak,Sepatu Sekolah Hitam W,Sepatu Sport merk Y,Sepatu Sport merk Z,Serum Vitamin,Shampo Anti Dandruff,Shampo Biasa,Stripe Pants,Sunblock Cream,Sweater Top Panjang,Tali Ban Ikat Pinggang,Tali Pinggang Anak,Tali Pinggang Gesper Pria,Tank Top,Tas Kosmetik,Tas Kulit Selempang,Tas Makeup,Tas Multifungsi,Tas Pinggang Wanita,Tas Ransel Mini,Tas Sekolah Anak Laki-laki,Tas Sekolah Anak Perempuan,Tas Tangan,Tas Travel,Tas Waist Bag,Wedges Hitam,Woman Ripped Jeans
kode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
#1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#10,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
#100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#1000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0


In [ ]:
#def encode_units(x):
#    if x <= 0:
#        return 0
#    if x >= 1:
#        return 1
#basket_sets = basket.applymap(encode_units)
#basket_sets.head())

In [26]:
# Build up the frequent items 
frequent = apriori(basket, min_support=10/len(df), max_len=3, use_colnames=True)
frequent

,support,itemsets
0,0.016232,(Atasan Baju Belang)
1,0.118841,(Atasan Kaos Putih)
2,0.380290,(Baju Batik Wanita)
3,0.121159,(Baju Kaos Anak - Karakter Kartun)
4,0.115942,(Baju Kaos Anak - Superheroes)
...,...,...
32527,0.001159,"(Tas Tangan, Wedges Hitam, Tas Travel)"
32528,0.012174,"(Tas Waist Bag, Wedges Hitam, Tas Travel)"
32529,0.000870,"(Tas Waist Bag, Woman Ripped Jeans , Tas Travel)"
32530,0.003478,"(Woman Ripped Jeans , Wedges Hitam, Tas Travel)"


In [27]:
# Create the rules
rules = association_rules(frequent, metric="lift", min_threshold=1)
rules.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Baju Kaos Anak - Karakter Kartun),(Atasan Baju Belang),0.121159,0.016232,0.002319,0.019139,1.179084,0.000352,1.002964
1,(Atasan Baju Belang),(Baju Kaos Anak - Karakter Kartun),0.016232,0.121159,0.002319,0.142857,1.179084,0.000352,1.025314
2,(Atasan Baju Belang),(Baju Kaos Olahraga),0.016232,0.089275,0.001739,0.107143,1.200139,0.000290,1.020012
3,(Baju Kaos Olahraga),(Atasan Baju Belang),0.089275,0.016232,0.001739,0.019481,1.200139,0.000290,1.003313
4,(Atasan Baju Belang),(Blouse Denim),0.016232,0.182319,0.003478,0.214286,1.175335,0.000519,1.040685


In [17]:
rules.sample(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
18580,(Baju Kaos Anak - Superheroes),"(Blouse Denim, Shampo Anti Dandruff)",0.115942,0.035652,0.006087,0.052500,1.472561,0.001953,1.017781
12378,(Obat Penumbuh Rambut),"(Baju Batik Wanita, Sunblock Cream)",0.068116,0.023478,0.002319,0.034043,1.449961,0.000720,1.010937
58964,"(Kaos, Celana Pendek Green/Hijau)",(Kuas Makeup ),0.031304,0.034203,0.001449,0.046296,1.353578,0.000379,1.012680
16811,(Sepatu Sport merk Z),"(Obat Penumbuh Rambut, Baju Kaos Anak - Karakt...",0.257391,0.008696,0.002609,0.010135,1.165541,0.000371,1.001454
35764,"(Tas Makeup, Dompet Card Holder)",(Baju Renang Pria Dewasa),0.009855,0.090725,0.001159,0.117647,1.296749,0.000265,1.030512
20332,"(Sepatu Kulit Casual, Baju Kaos Anak - Superhe...",(Hair Dryer),0.006667,0.237971,0.002609,0.391304,1.644336,0.001022,1.251905
44728,"(Blouse Denim, Tas Travel)",(Woman Ripped Jeans ),0.029855,0.034203,0.004928,0.165049,4.825572,0.003906,1.156710
69852,"(Shampo Biasa, Cream Whitening)",(Stripe Pants),0.035652,0.005507,0.000580,0.016260,2.952503,0.000383,1.010931
96715,(Tas Travel),"(Sunblock Cream, Wedges Hitam)",0.220870,0.017391,0.005217,0.023622,1.358268,0.001376,1.006381
88468,(Sepatu Sandal Anak),"(Wedges Hitam, Koper Fiber)",0.307826,0.015652,0.008696,0.028249,1.804771,0.003878,1.012963


In [18]:
rules_new = pd.DataFrame(rules[ (rules['lift'] >= 1) & (rules['confidence'] >= 0.5) ])

In [19]:
rules_new.sort_values(by=['lift'], ascending = False, inplace=True)

In [20]:
rules_new.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
69896,"(Stripe Pants, Cream Whitening)",(Woman Ripped Jeans ),0.000580,0.034203,0.000580,1.000000,29.237288,0.000560,inf
96420,"(Tank Top, Stripe Pants)",(Woman Ripped Jeans ),0.000580,0.034203,0.000580,1.000000,29.237288,0.000560,inf
32904,"(Tas Makeup, Tas Sekolah Anak Perempuan)",(Baju Renang Anak Perempuan),0.000870,0.035942,0.000870,1.000000,27.822581,0.000838,inf
32487,"(Baju Renang Anak Perempuan, Sepatu Sekolah Hi...",(Tas Makeup),0.001739,0.040870,0.001739,1.000000,24.468085,0.001668,inf
32888,"(Tas Makeup, Tas Pinggang Wanita)",(Baju Renang Anak Perempuan),0.011884,0.035942,0.010435,0.878049,24.429583,0.010008,7.905275
32914,"(Tas Makeup, Tas Travel)",(Baju Renang Anak Perempuan),0.012464,0.035942,0.010145,0.813953,22.646287,0.009697,5.181812
32355,"(Obat Penumbuh Rambut, Tas Makeup)",(Baju Renang Anak Perempuan),0.001449,0.035942,0.001159,0.800000,22.258065,0.001107,4.820290
88840,"(Stripe Pants, Koper Fiber)",(Tas Kulit Selempang),0.000870,0.031594,0.000580,0.666667,21.100917,0.000552,2.905217
32894,"(Tas Makeup, Tas Ransel Mini)",(Baju Renang Anak Perempuan),0.015362,0.035942,0.011304,0.735849,20.473220,0.010752,3.649648
56940,"(Celana Pendek Casual, Stripe Pants)",(Woman Ripped Jeans ),0.003768,0.034203,0.002609,0.692308,20.241199,0.002480,3.138841


In [21]:
#rules_new['rhs items'] = rules_new['consequents'].apply(lambda x:len(x))
#rules_new.head()

In [22]:
rules_product = pd.DataFrame(rules[ (rules['lift'] >= 1) & (rules['confidence'] >= 0.1) ])

In [28]:
#rules[rules.antecedents.apply(str).str.contains("101 Dalmatians")].sort_values('lift', ascending=False)
tasmakeup = rules_product[rules_product.consequents.apply(str).str.contains("Tas Makeup")].sort_values('lift', ascending=False)
tasmakeup.head(3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
32487,"(Baju Renang Anak Perempuan, Sepatu Sekolah Hi...",(Tas Makeup),0.001739,0.040870,0.001739,1.000000,24.468085,0.001668,inf
32889,(Baju Renang Anak Perempuan),"(Tas Makeup, Tas Pinggang Wanita)",0.035942,0.011884,0.010435,0.290323,24.429583,0.010008,1.392345
32915,(Baju Renang Anak Perempuan),"(Tas Makeup, Tas Travel)",0.035942,0.012464,0.010145,0.282258,22.646287,0.009697,1.375893


In [29]:
renangpria = rules_product[rules_product.consequents.apply(str).str.contains("Baju Renang Pria Anak-anak")].sort_values('lift', ascending=False)
renangpria.head(3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
34072,"(Tas Waist Bag, Gembok Koper)",(Baju Renang Pria Anak-anak),0.014783,0.009275,0.004058,0.274510,29.595588,0.003921,1.365593
33881,"(Gembok Koper, Flat Shoes Ballerina)",(Baju Renang Pria Anak-anak),0.021739,0.009275,0.004058,0.186667,20.125000,0.003856,1.218104
33266,"(Jeans Jumbo, Celana Jeans Sobek Wanita)",(Baju Renang Pria Anak-anak),0.045507,0.009275,0.005507,0.121019,13.047373,0.005085,1.127129
